In [1]:
%cd ../..

/home/matheus/Desktop/Itens/Projetos/llm2vec-embeddings-classification


In [22]:
import os
import numpy as np
import pandas as pd
from src.core.utils import read_json

# Configura o Pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

def load_results_to_dataframe(base_path: str) -> pd.DataFrame:
    """
    Load results from JSON files into a pandas DataFrame.
    """
    results = []
    
    # Traverse the directory structure
    for dataset_name in os.listdir(base_path):
        dataset_path = os.path.join(base_path, dataset_name)
        if os.path.isdir(dataset_path):
            for model_type in os.listdir(dataset_path):
                model_type_path = os.path.join(dataset_path, model_type)
                
                if os.path.isdir(model_type_path):
                    for model_name in os.listdir(model_type_path):
                        model_name_path = os.path.join(model_type_path, model_name)
                        
                        # Define paths based on whether prompt_name is needed
                        if model_type != "bert":
                            subdirs = [os.path.join(model_name_path, prompt) for prompt in os.listdir(model_name_path)]
                        else:
                            subdirs = [model_name_path]
                        
                        # Process results.json files from determined paths
                        for subdir in subdirs:
                            for classifier in os.listdir(subdir):
                                classifier_path = os.path.join(subdir, classifier)
                                
                                # Check for the results.json in the classifier path
                                json_file_path = os.path.join(classifier_path, 'results.json')
                                
                                if os.path.isfile(json_file_path):
                                    result_data = read_json(json_file_path)

                                    keys_to_extract = ["split0_test_f1_score", "split1_test_f1_score", "split2_test_f1_score", "split3_test_f1_score", "split4_test_f1_score", "mean_test_f1_score", "embedding_generation_time", "embedding_generation_size"]
    
                                    # Extrai apenas as chaves especificadas
                                    result_data= {key: result_data.get(key) for key in keys_to_extract}
                                    
                                    # Add metadata to the result data
                                    result_data['dataset_name'] = dataset_name
                                    result_data['model_type'] = model_type
                                    result_data['model_name'] = model_name
                                    result_data['classifier'] = classifier
                                    
                                    # Add prompt_name if applicable
                                    if model_type != "bert":
                                        result_data['prompt_name'] = os.path.basename(subdir)
                                    else:
                                        result_data['prompt_name'] = None
                                    
                                    results.append(result_data)

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    # Specify the order of the columns
    columns_first = ['dataset_name', 'model_type', 'model_name', 'classifier']
    if 'prompt_name' in results_df.columns:
        columns_first.append('prompt_name')
    column_order = columns_first + [col for col in results_df.columns if col not in columns_first]
    results_df = results_df[column_order]
    
    return results_df

In [24]:
base_path = 'results' 

df = load_results_to_dataframe(base_path)
df

,dataset_name,model_type,model_name,classifier,prompt_name,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,split4_test_f1_score,mean_test_f1_score,embedding_generation_time,embedding_generation_size
0,Dmoz-Science.csv,bert,sentence-transformers_all-distilroberta-v1,knn,None,0.781488,0.781423,0.807855,0.787449,0.798073,0.791258,4.378122,18432128
1,Dmoz-Science.csv,bert,sentence-transformers_all-MiniLM-L12-v2,knn,None,0.793738,0.784877,0.813567,0.790159,0.797160,0.795900,2.609955,9216128
2,Dmoz-Science.csv,bert,sentence-transformers_all-MiniLM-L6-v2,knn,None,0.790149,0.799200,0.819735,0.787873,0.797702,0.798932,2.709429,9216128
3,Dmoz-Science.csv,bert,sentence-transformers_all-mpnet-base-v2,knn,None,0.821972,0.812738,0.834973,0.811677,0.830549,0.822382,3.978258,18432128
4,Dmoz-Science.csv,llm2vec,McGill-NLP_LLM2Vec-Meta-Llama-3-8B-Instruct-mn...,knn,instruction_summary_prompt,0.802117,0.798172,0.828751,0.798943,0.820523,0.809701,88.551751,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,review_polarity.csv,llm2vec,McGill-NLP_LLM2Vec-Mistral-7B-Instruct-v2-mntp...,knn,instruction_classification_prompt,0.795752,0.779712,0.746347,0.781351,0.768741,0.774381,158.887130,128
106,review_polarity.csv,llm2vec,McGill-NLP_LLM2Vec-Mistral-7B-Instruct-v2-mntp...,knn,base_prompt,0.785244,0.779712,0.748410,0.781009,0.749738,0.768823,147.025850,128
107,review_polarity.csv,llm2vec,McGill-NLP_LLM2Vec-Sheared-LLaMA-mntp-supervised,knn,instruction_summary_prompt,0.763671,0.759978,0.729923,0.760221,0.709838,0.744726,50.659078,128
108,review_polarity.csv,llm2vec,McGill-NLP_LLM2Vec-Sheared-LLaMA-mntp-supervised,knn,instruction_classification_prompt,0.744483,0.747043,0.736201,0.751192,0.723348,0.740454,50.584882,128


# Desempenho

In [37]:
import pandas as pd

# Defina os splits que você quer combinar
splits = ['split0_test_f1_score', 
          'split1_test_f1_score', 
          'split2_test_f1_score', 
          'split3_test_f1_score', 
          'split4_test_f1_score']

# Agrupar pelo 'prompt_name' e combinar os splits em uma lista para cada 'prompt_name'
df_combined = df.groupby('model_type')[splits].apply(lambda x: x.values.flatten().tolist()).reset_index()

# Exibir o resultado
df_combined  # A coluna 0 contém a lista dos splits combinados


,model_type,0
0,bert,"[0.7814881599520914, 0.7814233885535803, 0.807..."
1,llm2vec,"[0.8021170140394833, 0.7981716507412795, 0.828..."


In [44]:
bert = df_combined[df_combined['model_type'] == "bert"][0].tolist()[0]
llm2vec = df_combined[df_combined['model_type'] == "llm2vec"][0].tolist()[0]

In [40]:
from scipy import stats

#Se o p-valor for maior que 0,05: Os dados não apresentam evidências suficientes para rejeitar a hipótese de normalidade, ou seja, os dados podem ser considerados normalmente distribuídos.
#Se o p-valor for menor que 0,05: Os dados provavelmente não seguem uma distribuição normal, o que pode indicar a necessidade de ajustes ou o uso de testes não paramétricos.

# Teste de Shapiro-Wilk para normalidade
shapiro_bert_stat, shapiro_bert_p = stats.shapiro(bert)
shapiro_llm2vec_stat, shapiro_llm2vec_p = stats.shapiro(llm2vec)

# Imprimir os resultados do teste de Shapiro-Wilk
print(f'Teste de Shapiro-Wilk para BERT: Estatística = {shapiro_bert_stat}, p-valor = {shapiro_bert_p}')
print(f'Teste de Shapiro-Wilk para LLM2Vec: Estatística = {shapiro_llm2vec_stat}, p-valor = {shapiro_llm2vec_p}')

Teste de Shapiro-Wilk para BERT: Estatística = 0.9267336662178758, p-valor = 3.263592024901407e-05
Teste de Shapiro-Wilk para LLM2Vec: Estatística = 0.9744571322962526, p-valor = 4.350122302301677e-07


In [45]:
from scipy.stats import kruskal

# Executando o teste de Kruskal-Wallis
stat, p_value = kruskal(bert, llm2vec)

# Exibindo o resultado
print(f"Estatística de Kruskal-Wallis: {stat}")
print(f"Valor-p: {p_value}")

Estatística de Kruskal-Wallis: 8.349214935625344
Valor-p: 0.003858540119644899


In [46]:
from scipy.stats import rankdata
import numpy as np

# Cálculo das medianas para BERT e LLM2Vec
mediana_bert = np.median(bert)
mediana_llm2vec = np.median(llm2vec)

print(f'Mediana BERT: {mediana_bert}')
print(f'Mediana LLM2Vec: {mediana_llm2vec}')


Mediana BERT: 0.7890110456523249
Mediana LLM2Vec: 0.7987677932179514


# Prompt

In [ ]:
import pandas as pd

# Defina os splits que você quer combinar
splits = ['split0_test_f1_score', 
          'split1_test_f1_score', 
          'split2_test_f1_score', 
          'split3_test_f1_score', 
          'split4_test_f1_score']

# Agrupar pelo 'prompt_name' e combinar os splits em uma lista para cada 'prompt_name'
df_combined = df.groupby('prompt_name')[splits].apply(lambda x: x.values.flatten().tolist()).reset_index()

# Exibir o resultado
df_combined  # A coluna 0 contém a lista dos splits combinados


,prompt_name,0
0,base_prompt,"[0.8041448892389337, 0.8074692058976428, 0.847..."
1,instruction_classification_prompt,"[0.7991820588375381, 0.7924220543974968, 0.830..."
2,instruction_summary_prompt,"[0.8021170140394833, 0.7981716507412795, 0.828..."


In [17]:
base_prompt = df_combined[df_combined['prompt_name'] == "base_prompt"][0].tolist()[0]
instruction_summary_prompt = df_combined[df_combined['prompt_name'] == "instruction_summary_prompt"][0].tolist()[0]
instruction_classification_prompt = df_combined[df_combined['prompt_name'] == "instruction_classification_prompt"][0].tolist()[0]

In [ ]:
from scipy.stats import shapiro

#Se o p-valor for maior que 0,05: Os dados não apresentam evidências suficientes para rejeitar a hipótese de normalidade, ou seja, os dados podem ser considerados normalmente distribuídos.
#Se o p-valor for menor que 0,05: Os dados provavelmente não seguem uma distribuição normal, o que pode indicar a necessidade de ajustes ou o uso de testes não paramétricos.

# Teste de Shapiro-Wilk
shapiro_base = shapiro(base_prompt)
shapiro_summary = shapiro(instruction_summary_prompt)
shapiro_classification = shapiro(instruction_classification_prompt)

print("Shapiro-Wilk:")
print("base_prompt:", shapiro_base)
print("instruction_summary_prompt:", shapiro_summary)
print("instruction_classification_prompt:", shapiro_classification)

Shapiro-Wilk:
base_prompt: ShapiroResult(statistic=np.float64(0.9780683841149544), pvalue=np.float64(0.016842197234145814))
instruction_summary_prompt: ShapiroResult(statistic=np.float64(0.9658230116257912), pvalue=np.float64(0.0008753770107978666))
instruction_classification_prompt: ShapiroResult(statistic=np.float64(0.9668374884912305), pvalue=np.float64(0.0011021261106809728))


In [21]:
from scipy.stats import kruskal

# Executando o teste de Kruskal-Wallis
stat, p_value = kruskal(base_prompt, instruction_summary_prompt, instruction_classification_prompt)

# Exibindo o resultado
print(f"Estatística de Kruskal-Wallis: {stat}")
print(f"Valor-p: {p_value}")

Estatística de Kruskal-Wallis: 1.0434580723085982
Valor-p: 0.5934934886155303


# Tempo

In [28]:
bert = df[df['model_type'] == 'bert']['embedding_generation_time']
llm2vec = df[df['model_type'] == 'llm2vec']['embedding_generation_time']

In [29]:
from scipy import stats

#Se o p-valor for maior que 0,05: Os dados não apresentam evidências suficientes para rejeitar a hipótese de normalidade, ou seja, os dados podem ser considerados normalmente distribuídos.
#Se o p-valor for menor que 0,05: Os dados provavelmente não seguem uma distribuição normal, o que pode indicar a necessidade de ajustes ou o uso de testes não paramétricos.

# Teste de Shapiro-Wilk para normalidade
shapiro_bert_stat, shapiro_bert_p = stats.shapiro(bert)
shapiro_llm2vec_stat, shapiro_llm2vec_p = stats.shapiro(llm2vec)

# Imprimir os resultados do teste de Shapiro-Wilk
print(f'Teste de Shapiro-Wilk para BERT: Estatística = {shapiro_bert_stat}, p-valor = {shapiro_bert_p}')
print(f'Teste de Shapiro-Wilk para LLM2Vec: Estatística = {shapiro_llm2vec_stat}, p-valor = {shapiro_llm2vec_p}')

Teste de Shapiro-Wilk para BERT: Estatística = 0.6772026339835802, p-valor = 2.0822588366427486e-05
Teste de Shapiro-Wilk para LLM2Vec: Estatística = 0.7480545454568027, p-valor = 3.9981888472766954e-11


In [30]:
from scipy.stats import kruskal

# Executando o teste de Kruskal-Wallis
stat, p_value = kruskal(bert, llm2vec)

# Exibindo o resultado
print(f"Estatística de Kruskal-Wallis: {stat}")
print(f"Valor-p: {p_value}")

Estatística de Kruskal-Wallis: 47.89483483483485
Valor-p: 4.4970478603666605e-12


In [34]:
from scipy.stats import rankdata
import numpy as np

# Cálculo das medianas para BERT e LLM2Vec
mediana_bert = np.median(bert)
mediana_llm2vec = np.median(llm2vec)

print(f'Mediana BERT: {mediana_bert}')
print(f'Mediana LLM2Vec: {mediana_llm2vec}')

# Comparando as medianas
if mediana_bert < mediana_llm2vec:
    print("O modelo BERT tem os menores valores de tempo de geração de embeddings.")
else:
    print("O modelo LLM2Vec tem os menores valores de tempo de geração de embeddings.")


Mediana BERT: 4.344724655151367
Mediana LLM2Vec: 135.5660218000412
O modelo BERT tem os menores valores de tempo de geração de embeddings.
